In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd 'gdrive/My Drive/Colab Notebooks/Udacity_DLND/RGB_to_GRAYSCALE_Autoencoder-'
  
!pwd


/content/gdrive/My Drive/Colab Notebooks/Udacity_DLND/RGB_to_GRAYSCALE_Autoencoder-
/content/gdrive/My Drive/Colab Notebooks/Udacity_DLND/RGB_to_GRAYSCALE_Autoencoder-


In [3]:
import torch
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import cv2

# np.set_printoptions(threshold=np.inf)

In [ ]:
from glob import glob

np.random.seed(313)

# load filenames for color and gray images
color_files = np.array(glob("color_images/*.jpg"))
gray_files = np.array(glob("gray_images/*.jpg"))

# print number of images in each dataset
print('There are %d total color images.' % len(color_files))
print('There are %d total gray images.' % len(gray_files))

In [0]:
# num_images = 3670
num_images = len(color_files)

In [ ]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [0]:
# dataset = []

# for i in range(1, num_images+1):
#     img = cv2.imread("color_images/color_" +str(i) +".jpg" )
#     dataset.append(np.array(img))

# dataset_source = np.asarray(dataset)
# print(dataset_source.shape)

# dataset_tar = []

# for i in range(1, num_images+1):
#     img = cv2.imread("gray_images/gray_" +str(i) +".jpg", 0)    
#     dataset_tar.append(np.array(img))

# dataset_target = np.asarray(dataset_tar)
# print(dataset_target.shape)


## Reference: https://stackoverflow.com/questions/45099554/how-to-simplify-dataloader-for-autoencoder-in-pytorch

import PIL
class ImageLoader(torch.utils.data.Dataset):
  
  def __init__(self, source_root, target_root, source_tform=None, target_tform=None, imgloader=PIL.Image.open):
      super(ImageLoader, self).__init__()

      self.source_root = source_root
      self.target_root = target_root

      self.source_filenames = sorted(glob(source_root))
      self.target_filenames = sorted(glob(target_root))

      self.source_tform = source_tform
      self.target_tform = target_tform

      self.imgloader = imgloader

  def __len__(self):
      return len(self.source_filenames)

  def __getitem__(self, i):
      source, target = self.imgloader(self.source_filenames[i]), self.imgloader(self.target_filenames[i])  # io.imread(self.filenames[i])
      
      if self.source_tform:
          source = self.source_tform(source)
      
      if self.target_tform:
          target = self.target_tform(target)
      
      return source, target


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
dataset_target = dataset_target[:, :, :, np.newaxis]
dataset_target.shape

(3670, 128, 128, 1)

In [0]:
def autoencoder(inputs): # Undercomplete Autoencoder
    
    # Encoder
    
    net = tf.layers.conv2d(inputs, 128, 2, activation = tf.nn.relu)
    print(net.shape)
    net = tf.layers.max_pooling2d(net, 2, 2, padding = 'same')
    print(net.shape)

    # Decoder
    
    net = tf.image.resize_nearest_neighbor(net, tf.constant([129, 129]))
    net = tf.layers.conv2d(net, 1, 2, activation = None, name = 'outputOfAuto')

    print(net.shape)
    
    return net

In [5]:
ae_inputs = tf.placeholder(tf.float32, (None, 128, 128, 3), name = 'inputToAuto')
ae_target = tf.placeholder(tf.float32, (None, 128, 128, 1))

ae_outputs = autoencoder(ae_inputs)
lr = 0.001

loss = tf.reduce_mean(tf.square(ae_outputs - ae_target))
train_op = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)
# Intialize the network 
init = tf.global_variables_initializer()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
(?, 127, 127, 128)
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
(?, 64, 64, 128)
(?, 128, 128, 1)


#### If you don't want to train the network skip the cell righ below and dowload the pre-trained model. After downloading the pre-trained model run the cell below to the immediate below cell. 

In [0]:
batch_size = 32
epoch_num = 50

saving_path = 'SavedModel/AutoencoderColorToGray.ckpt'

saver_ = tf.train.Saver(max_to_keep = 3)

batch_img = dataset_source[0:batch_size]
batch_out = dataset_target[0:batch_size]

num_batches = num_images//batch_size

sess = tf.Session()
sess.run(init)

for ep in range(epoch_num):
    batch_size = 0
    for batch_n in range(num_batches): # batches loop

        _, c = sess.run([train_op, loss], feed_dict = {ae_inputs: batch_img, ae_target: batch_out})
        print("Epoch: {} - cost = {:.5f}" .format((ep+1), c))
            
        batch_img = dataset_source[batch_size: batch_size+32]
        batch_out = dataset_target[batch_size: batch_size+32]
            
        batch_size += 32
    
    saver_.save(sess, saving_path, global_step = ep)
recon_img = sess.run([ae_outputs], feed_dict = {ae_inputs: batch_img})

sess.close()

Epoch: 1 - cost = 28606.42188
Epoch: 1 - cost = 23159.11523
Epoch: 1 - cost = 17017.81836
Epoch: 1 - cost = 17763.41797
Epoch: 1 - cost = 12389.10547
Epoch: 1 - cost = 8201.01367
Epoch: 1 - cost = 4748.31641
Epoch: 1 - cost = 2944.66992
Epoch: 1 - cost = 1484.49780
Epoch: 1 - cost = 836.98547
Epoch: 1 - cost = 468.59363
Epoch: 1 - cost = 686.64954
Epoch: 1 - cost = 982.24829
Epoch: 1 - cost = 1909.65649
Epoch: 1 - cost = 2068.51440
Epoch: 1 - cost = 2603.14868
Epoch: 1 - cost = 2456.30713
Epoch: 1 - cost = 2391.12524
Epoch: 1 - cost = 2319.79639
Epoch: 1 - cost = 1776.34314
Epoch: 1 - cost = 1691.51733
Epoch: 1 - cost = 1006.02887
Epoch: 1 - cost = 854.51550
Epoch: 1 - cost = 666.13800
Epoch: 1 - cost = 531.55920
Epoch: 1 - cost = 487.99133
Epoch: 1 - cost = 337.07318
Epoch: 1 - cost = 382.10175
Epoch: 1 - cost = 295.61420
Epoch: 1 - cost = 333.11765
Epoch: 1 - cost = 360.19092
Epoch: 1 - cost = 349.62802
Epoch: 1 - cost = 456.58890
Epoch: 1 - cost = 448.68921
Epoch: 1 - cost = 367.637

In [0]:
saver = tf.train.Saver()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver.restore(sess, 'SavedModel/AutoencoderColorToGray.ckpt-49')

INFO:tensorflow:Restoring parameters from SavedModel/AutoencoderColorToGray.ckpt-49


In [0]:
import glob as gl 

filenames = gl.glob('flower_images/*.png')

test_data = []
for file in filenames[0:100]:
    test_data.append(np.array(cv2.imread(file)))

test_dataset = np.asarray(test_data)
print(test_dataset.shape)

# Running the test data on the autoencoder
batch_imgs = test_dataset
gray_imgs = sess.run(ae_outputs, feed_dict = {ae_inputs: batch_imgs})

(0,)


ValueError: ignored

In [0]:
print(gray_imgs.shape)

for i in range(gray_imgs.shape[0]):
    cv2.imwrite('gen_gray_images/gen_gray_' +str(i) +'.jpeg', gray_imgs[i])

NameError: ignored